In [1]:
work_dir = "/home/spaka002/NSF_REU_2024/"

In [2]:
from notebooks.utilities.STC_grid_search import *

import pandas as pd
import numpy as np

from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.ensemble import RandomForestClassifier

In [3]:
data_folder = f"{work_dir}classification_datasets/"

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [5]:
# device = 'cpu'

In [6]:
model = RandomForestClassifier(random_state = 18)

param_dict = {'max_depth':[2, 3, 4, 5, 6, 10, None],
              'max_features': ['sqrt', 'log2', 1, 3, 5, 7, 9, None],
              'n_estimators': [5, 10, 25, 50, 100],
              'min_samples_split': [2, 4, 8, 16, 32, 64, 128, 256]}

In [7]:
total_cells = 1
for s in param_dict.values():
    total_cells *= len(s)

print(f"{total_cells} total combinations.")

2240 total combin.


In [14]:
portion_of_combinations = 0.05

cv_splits = 5

data = np.array(pd.read_csv(f"{data_folder}alzheimers_disease_data.csv"))

x = data[:, :-1]
y = data[:, -1]

del data


x, y = get_subset(x = x, y = y, portion = 1.0, random_state = 18)

print(f"X Shape: {x.shape}; Y Shape: {y.shape}")

X Shape: (2149, 32); Y Shape: (2149,)


In [15]:
best_estimated_params= return_best_k_params(model = model, 
                                            param_dict = param_dict,
                                            X = x,
                                            Y = y,
                                            num_top_combinations = 15,
                                            cv_splits = 5,
                                            portion_of_combinations = 0.05,
                                            STC_model_type = 'costco',
                                            rank = 25,
                                            device = device,
                                            verbose = True)

112/2240 total combinations in sparse tensor.
1/112 param_combinations done.
2/112 param_combinations done.
3/112 param_combinations done.
4/112 param_combinations done.
5/112 param_combinations done.
6/112 param_combinations done.
7/112 param_combinations done.
8/112 param_combinations done.
9/112 param_combinations done.
10/112 param_combinations done.
11/112 param_combinations done.
12/112 param_combinations done.
13/112 param_combinations done.
14/112 param_combinations done.
15/112 param_combinations done.
16/112 param_combinations done.
17/112 param_combinations done.
18/112 param_combinations done.
19/112 param_combinations done.
20/112 param_combinations done.
21/112 param_combinations done.
22/112 param_combinations done.
23/112 param_combinations done.
24/112 param_combinations done.
25/112 param_combinations done.
26/112 param_combinations done.
27/112 param_combinations done.
28/112 param_combinations done.
29/112 param_combinations done.
30/112 param_combinations done.
31/

In [16]:
for x in best_estimated_params:
    print(f"{x[0]} \nactual eval = {x[1]:.4f}\n")

{'max_depth': 6, 'max_features': None, 'n_estimators': 10, 'min_samples_split': 4} 
actual eval = 0.9248

{'max_depth': 6, 'max_features': None, 'n_estimators': 100, 'min_samples_split': 32} 
actual eval = 0.9242

{'max_depth': 10, 'max_features': None, 'n_estimators': 100, 'min_samples_split': 32} 
actual eval = 0.9242

{'max_depth': 6, 'max_features': None, 'n_estimators': 10, 'min_samples_split': 32} 
actual eval = 0.9231

{'max_depth': None, 'max_features': None, 'n_estimators': 10, 'min_samples_split': 32} 
actual eval = 0.9231

{'max_depth': 10, 'max_features': None, 'n_estimators': 10, 'min_samples_split': 32} 
actual eval = 0.9225

{'max_depth': 5, 'max_features': None, 'n_estimators': 10, 'min_samples_split': 32} 
actual eval = 0.9224

{'max_depth': 6, 'max_features': None, 'n_estimators': 10, 'min_samples_split': 8} 
actual eval = 0.9218

{'max_depth': 6, 'max_features': None, 'n_estimators': 5, 'min_samples_split': 32} 
actual eval = 0.9162

{'max_depth': 10, 'max_features':

In [17]:
actual_grid_search = torch.load(f"{work_dir}meta_datasets/RF_Alzheimers_805.pt")

In [18]:
param_list = list(param_dict)

# asked perplexity.ai for help with this
values, indices = torch.topk(actual_grid_search.flatten(), 10)

# Convert flat indices back to 3D indices
top_k_indices = np.array(np.unravel_index(indices.numpy(), actual_grid_search.shape)).T

# If you want to pair each value with its 3D index
for value, index in zip(values, top_k_indices):
    print(f"Value: {value.item():.4f}, Index: {tuple(index)}")

Value: 0.9345, Index: (4, 2, 3, 7)
Value: 0.9345, Index: (4, 1, 3, 7)
Value: 0.9344, Index: (4, 0, 3, 7)
Value: 0.9325, Index: (4, 3, 3, 7)
Value: 0.9323, Index: (4, 3, 6, 6)
Value: 0.9309, Index: (3, 2, 3, 7)
Value: 0.9309, Index: (3, 1, 3, 7)
Value: 0.9303, Index: (3, 0, 3, 7)
Value: 0.9297, Index: (3, 3, 3, 7)
Value: 0.9287, Index: (4, 0, 5, 6)
